# Config

In [ ]:
from pathlib import Path

class Config:
    base_path = Path('/kaggle/input/MABe-mouse-behavior-detection')
    ann_path = base_path / 'train_annotation'
    train_track_path = base_path / 'train_tracking'
    test_track_path = base_path / 'test_tracking'
    train_metadata_path = base_path / 'train.csv'
    test_metadata_path = base_path / 'test.csv'
    RUN_TRAIN_DATA = False
    n_splits = 5

cfg = Config()

# Data preparation

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from  tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

train_metadata = pd.read_csv(cfg.train_metadata_path)
test_metadata = pd.read_csv(cfg.test_metadata_path)

def create_mouse_features(mouse_data, suffix):
    if len(mouse_data) == 0:
        return {}
    
    features = {}
    
    features[f'mean_x_{suffix}'] = mouse_data['x'].mean()
    features[f'mean_y_{suffix}'] = mouse_data['y'].mean()
    features[f'std_x_{suffix}'] = mouse_data['x'].std()
    features[f'std_y_{suffix}'] = mouse_data['y'].std()
    
    mouse_data = mouse_data.sort_values('video_frame')
    dx = mouse_data['x'].diff()
    dy = mouse_data['y'].diff()
    speed = np.sqrt(dx**2 + dy**2)
    
    features[f'speed_mean_{suffix}'] = speed.mean()
    features[f'speed_std_{suffix}'] = speed.std()
    features[f'speed_max_{suffix}'] = speed.max()
    features[f'total_distance_{suffix}'] = speed.sum()
    
    acceleration = speed.diff()
    features[f'accel_mean_{suffix}'] = acceleration.mean()
    features[f'accel_std_{suffix}'] = acceleration.std()
    
    return features

def create_interaction_features(agent_data, target_data, agent_id, target_id):
    
    if len(agent_data) == 0 or len(target_data) == 0:
        return None
    
    merged = pd.merge(agent_data, target_data, on='video_frame', 
                     suffixes=('_agent', '_target'), how='inner')
    
    if len(merged) == 0:
        return None
    
    features = {}
    
    agent_features = create_mouse_features(agent_data, 'agent')
    features.update(agent_features)
    
    target_features = create_mouse_features(target_data, 'target')
    features.update(target_features)
    
    merged['distance'] = np.sqrt(
        (merged['x_agent'] - merged['x_target'])**2 + 
        (merged['y_agent'] - merged['y_target'])**2
    )
    
    agent_speed = np.sqrt(merged['x_agent'].diff()**2 + merged['y_agent'].diff()**2)
    target_speed = np.sqrt(merged['x_target'].diff()**2 + merged['y_target'].diff()**2)
    
    features['distance_mean'] = merged['distance'].mean()
    features['distance_std'] = merged['distance'].std()
    features['distance_min'] = merged['distance'].min()
    features['distance_max'] = merged['distance'].max()
    
    features['relative_speed_mean'] = (agent_speed - target_speed).abs().mean()
    features['speed_ratio'] = agent_speed.mean() / (target_speed.mean() + 1e-8)
    
    if len(merged) > 1:
        agent_dx = merged['x_agent'].iloc[-1] - merged['x_agent'].iloc[0]
        agent_dy = merged['y_agent'].iloc[-1] - merged['y_agent'].iloc[0]
        target_dx = merged['x_target'].iloc[-1] - merged['x_target'].iloc[0]
        target_dy = merged['y_target'].iloc[-1] - merged['y_target'].iloc[0]
        
        features['agent_movement'] = np.sqrt(agent_dx**2 + agent_dy**2)
        features['target_movement'] = np.sqrt(target_dx**2 + target_dy**2)
    
    features['agent_id'] = agent_id
    features['target_id'] = target_id
    features['frames_count'] = len(merged)
    
    return features

def prepare_train_dataset(segment_size=50):
    
    all_features = []
    all_labels = []
    
    for lab_id in tqdm(os.listdir(cfg.ann_path)):
        lab_ann_path = cfg.ann_path / lab_id
        lab_track_path = cfg.train_track_path / lab_id
        
        if not lab_ann_path.exists() or not lab_track_path.exists():
            continue
            
        for ann_file in tqdm(os.listdir(lab_ann_path), desc=lab_id):
            if ann_file.endswith('.parquet'):
                video_id = ann_file.replace('.parquet', '')
                
                try:
                    ann_df = pd.read_parquet(lab_ann_path / ann_file)
                    track_df = pd.read_parquet(lab_track_path / ann_file)
                    
                    annotated_segments = set()
                    for _, ann in ann_df.iterrows():
                        start, stop = ann['start_frame'], ann['stop_frame']
                        for seg_start in range(start, stop + 1, segment_size):
                            seg_end = min(seg_start + segment_size - 1, stop)
                            if seg_end - seg_start >= 10:
                                annotated_segments.add((seg_start, seg_end, ann['agent_id'], ann['target_id'], ann['action']))
                    
                    for seg_start, seg_end, agent_id, target_id, action in annotated_segments:
                        segment_data = track_df[
                            (track_df['video_frame'] >= seg_start) & 
                            (track_df['video_frame'] <= seg_end)
                        ]
                        
                        agent_data = segment_data[segment_data['mouse_id'] == agent_id]
                        target_data = segment_data[segment_data['mouse_id'] == target_id]
                        
                        features = create_interaction_features(agent_data, target_data, agent_id, target_id)
                        
                        if features is not None:
                            features['video_id'] = video_id
                            features['start_frame'] = seg_start
                            features['stop_frame'] = seg_end
                            features['segment_size'] = seg_end - seg_start + 1
                            
                            all_features.append(features)
                            all_labels.append(action)
                    
                    all_frames = sorted(track_df['video_frame'].unique())
                    annotated_frames = set()
                    
                    for seg_start, seg_end, _, _, _ in annotated_segments:
                        annotated_frames.update(range(seg_start, seg_end + 1))
                    
                    for i in range(0, len(all_frames), segment_size):
                        if i + segment_size > len(all_frames):
                            break
                            
                        seg_start = all_frames[i]
                        seg_end = all_frames[min(i + segment_size - 1, len(all_frames) - 1)]
                        
                        segment_frames = set(range(seg_start, seg_end + 1))
                        if segment_frames & annotated_frames:
                            continue
                        
                        segment_data = track_df[
                            (track_df['video_frame'] >= seg_start) & 
                            (track_df['video_frame'] <= seg_end)
                        ]
                        
                        mice = segment_data['mouse_id'].unique()
                        if len(mice) == 0:
                            continue
                            
                        for agent in mice[:2]:  
                            for target in mice[:2]:
                                if agent == target:  
                                    agent_data = segment_data[segment_data['mouse_id'] == agent]
                                    target_data = segment_data[segment_data['mouse_id'] == target]
                                    
                                    features = create_interaction_features(agent_data, target_data, agent, target)
                                    
                                    if features is not None:
                                        features['video_id'] = video_id
                                        features['start_frame'] = seg_start
                                        features['stop_frame'] = seg_end
                                        features['segment_size'] = seg_end - seg_start + 1
                                        
                                        all_features.append(features)
                                        all_labels.append('no_action')
                        
                except Exception as e:
                    print(f"Video {video_id} error: {e}")
                    continue
    
    features_df = pd.DataFrame(all_features).fillna(0)
    
    return features_df, pd.Series(all_labels)

def prepare_test_dataset(segment_size=50):
    
    all_test_features = []
    
    
    for lab_id in os.listdir(cfg.test_track_path):
        lab_track_path = cfg.test_track_path / lab_id
        
        for track_file in os.listdir(lab_track_path):
            if track_file.endswith('.parquet'):
                video_id = track_file.replace('.parquet', '')
                
                try:
                    track_df = pd.read_parquet(lab_track_path / track_file)
                    all_frames = sorted(track_df['video_frame'].unique())
                    
                    for i in range(0, len(all_frames), segment_size):
                        seg_start = all_frames[i]
                        seg_end = all_frames[min(i + segment_size - 1, len(all_frames) - 1)]
                        
                        if seg_end - seg_start < 10:
                            continue
                            
                        segment_data = track_df[
                            (track_df['video_frame'] >= seg_start) & 
                            (track_df['video_frame'] <= seg_end)
                        ]
                        
                        mice = segment_data['mouse_id'].unique()
                        if len(mice) == 0:
                            continue
                        
                        for agent in mice:
                            for target in mice:
                                agent_data = segment_data[segment_data['mouse_id'] == agent]
                                target_data = segment_data[segment_data['mouse_id'] == target]
                                
                                features = create_interaction_features(agent_data, target_data, agent, target)
                                
                                if features is not None:
                                    features['video_id'] = video_id
                                    features['start_frame'] = seg_start
                                    features['stop_frame'] = seg_end
                                    features['segment_size'] = seg_end - seg_start + 1
                                    
                                    all_test_features.append(features)
                    
                        
                except Exception as e:
                    print(f"Video {video_id} error: {e}")
                    continue
    
    test_features_df = pd.DataFrame(all_test_features).fillna(0)
    
    return test_features_df

if cfg.RUN_TRAIN_DATA:
    train_features, train_labels = prepare_train_dataset(segment_size=50)
else:
    train_features = pd.read_parquet('/kaggle/input/mice-dataset/train_features.parquet')
    train_labels = pd.read_csv('/kaggle/input/mice-dataset/train_labels.csv')

test_features = prepare_test_dataset(segment_size=50)

# Training

In [ ]:
from sklearn.model_selection import GroupKFold
from sklearn.metrics import f1_score
import catboost as ctb

drop_cols = ['video_id']

scores = []

cv = GroupKFold(n_splits=cfg.n_splits)
models = []
for i, (tr_idx, val_idx) in enumerate(cv.split(train_features, groups=train_features['video_id'])):
    X_tr, X_val = train_features.iloc[tr_idx], train_features.iloc[val_idx]
    y_tr, y_val = train_labels.iloc[tr_idx], train_labels.iloc[val_idx]
    model = ctb.CatBoostClassifier(verbose=100, task_type='GPU', random_state=0, eval_metric='TotalF1:average=Macro', cat_features=['target_id', 'agent_id'])
    model.fit(X_tr.drop(columns=drop_cols), y_tr, eval_set=(X_val.drop(columns=drop_cols), y_val), early_stopping_rounds=50,)
    models.append(model)
    scores.append(f1_score(y_val, model.predict(X_val.drop(columns=drop_cols)), average='macro'))

print("-" * 20)
print("CV Score")
print("-" * 20)
print(f"{np.mean(scores):.4f} ± {np.std(scores):.4f}")

# Inference

In [ ]:
test_preds = []
for i in range(cfg.n_splits):
    test_preds.append(models[i].predict_proba(test_features[models[i].feature_names_]))
test_preds = np.mean(test_preds, 0)
preds = [models[0].classes_[x] for x in test_preds.argmax(1)]
max_prob = np.array([test_preds[i][x] for i, x in enumerate(test_preds.argmax(1))])
test_features['action'] = preds
test_features = test_features[(test_features['action'] != 'no_action') & (max_prob > 0.5)]
test_features['row_id'] = range(len(test_features))
test_features['agent_id'] = ["mouse{}".format(agent) for agent in test_features['agent_id'].values]
test_features['target_id'] = ["mouse{}".format(target) for target in test_features['target_id'].values]
test_features.loc[test_features['agent_id'] == test_features['target_id'], 'target_id'] = 'self'
test_features[['row_id', 'video_id', 'agent_id', 'target_id',
      'action', 'start_frame', 'stop_frame']].to_csv('submission.csv', index=False)